In [48]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Subset

In [49]:
data = np.load('./Mnist_10000._samples.npy')

In [50]:
# Convert the structured array to PyTorch tensors
images = torch.tensor([item['image'] for item in data], dtype=torch.float32)
labels = torch.tensor([item['label'] for item in data], dtype=torch.long)

# Flatten the images for a simple fully connected network
images = images.view(images.shape[0], -1)


# Create a dataset and data loader
dataset = TensorDataset(images, labels)


In [51]:
class_indices = {k: [] for k in range(10)}  # Assuming 10 classes (0-9)
for idx, (image, label) in enumerate(dataset):
    class_indices[label.item()].append(idx)

# Step 2: Randomly select 10 samples from each class
labeled_indices = []
for indices in class_indices.values():
    labeled_indices.extend(np.random.choice(indices, 10, replace=False))

# Create a mask for the rest of the data for testing
mask = np.ones(len(dataset), dtype=bool)
mask[labeled_indices] = False
unlabeled_indices = np.arange(len(dataset))[mask]

# Step 3: Create training and testing subsets
labeled_set = Subset(dataset, labeled_indices)
unlabeled_set = Subset(dataset, unlabeled_indices)

In [52]:
# Step 4: Create DataLoaders
labeledLoader = DataLoader(labeled_set, batch_size=10, shuffle=True)
unlabeledLoader = DataLoader(unlabeled_set, batch_size=50, shuffle=False)


In [53]:
class Net(nn.Module):

    def __init__(self):

        super(Net,self).__init__()

        self.fullyConnectedLayer = nn.Sequential(
            nn.Linear(784, 200),
            nn.ReLU(),
            nn.Linear(200,10)
        )

    def forward(self, input):

        output = self.fullyConnectedLayer(input)
        activatedOutput = F.log_softmax(output, dim = 1)

        return activatedOutput
    

In [54]:
#Create entropy loss for the unlabeled data 

def entropy_loss(logits):
    p = F.softmax(logits, dim=1)
    log_p = F.log_softmax(logits, dim=1)
    loss = -torch.sum(p * log_p, dim=1).mean()
    return loss

In [55]:
model1 = Net()

In [56]:
import torch.optim as optim

optimizer1 = optim.SGD(model1.parameters(), lr=0.03)
optimizer2 = optim.SGD(model1.parameters(), lr = 0.001)
num_epochs = 10
loss = 0
unsupervised_loss = 0

for epoch in range(num_epochs):
    model1.train()
    
    # Train on labeled data
    for images, labels in labeledLoader:
        optimizer1.zero_grad()
        outputs = model1(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer1.step()

    print(f"The supervised loss for epoch{epoch} is:{loss}")
    
    # Train on unlabeled data
    for images, labels in unlabeledLoader:
        optimizer2.zero_grad()
        outputs = model1(images)
        unsupervised_loss = entropy_loss(outputs)
        unsupervised_loss.backward()
        optimizer2.step()

    
    print(f"The un-supervised loss for epoch{epoch} is:{unsupervised_loss}")

The supervised loss for epoch0 is:2.289170265197754
The un-supervised loss for epoch0 is:2.3019609451293945
The supervised loss for epoch1 is:2.2965118885040283
The un-supervised loss for epoch1 is:2.3019821643829346
The supervised loss for epoch2 is:2.3038361072540283
The un-supervised loss for epoch2 is:2.302002191543579
The supervised loss for epoch3 is:2.2853522300720215
The un-supervised loss for epoch3 is:2.3020195960998535
The supervised loss for epoch4 is:2.2933411598205566
The un-supervised loss for epoch4 is:2.302036762237549
The supervised loss for epoch5 is:2.276184558868408
The un-supervised loss for epoch5 is:2.3020517826080322
The supervised loss for epoch6 is:2.286893129348755
The un-supervised loss for epoch6 is:2.302065372467041
The supervised loss for epoch7 is:2.2815845012664795
The un-supervised loss for epoch7 is:2.302081346511841
The supervised loss for epoch8 is:2.253382682800293
The un-supervised loss for epoch8 is:2.3020942211151123
The supervised loss for epo

In [57]:
def calculate_accuracy(loader,model):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  

    return 100 * correct / total

train_accuracy = calculate_accuracy(labeledLoader, model1)
test_accuracy = calculate_accuracy(unlabeledLoader, model1)

print(f'Training Accuracy: {train_accuracy}%')
print(f'Testing Accuracy: {test_accuracy}%')

Training Accuracy: 24.0%
Testing Accuracy: 10.666666666666666%


### Entropy minimization for Two Moons Dataset

In [68]:
selected_samples = np.load('./selected_samples.npy')

remaining_samples = np.load('./remaining_dataset.npy')

# Converting the selected samples and remaining samples into PyTorch tensors
selected_samples_tensor = torch.tensor(selected_samples, dtype=torch.float32)
remaining_samples_tensor = torch.tensor(remaining_samples, dtype=torch.float32)

# Extracting features and labels for both datasets
features_selected = selected_samples_tensor[:, :2]
labels_selected = selected_samples_tensor[:, 2].long()  # converting labels to long for classification

features_remaining = remaining_samples_tensor[:, :2]
labels_remaining = remaining_samples_tensor[:, 2].long()

# Creating TensorDatasets
selected_dataset = TensorDataset(features_selected, labels_selected)
remaining_dataset = TensorDataset(features_remaining, labels_remaining)

# Creating DataLoaders
selected_loader = DataLoader(selected_dataset, batch_size=1)  # small batch size for the small dataset
remaining_loader = DataLoader(remaining_dataset, batch_size=10)  # larger batch size for the larger dataset

In [69]:
class TwoMoonsNet(nn.Module):

    def __init__(self):

        super(TwoMoonsNet,self).__init__()

        self.fullyConnectedLayer = nn.Sequential(
            nn.Linear(2, 10),
            nn.ReLU(),
            nn.Linear(10,2)
        )

    def forward(self, input):

        output = self.fullyConnectedLayer(input)
        activatedOutput = F.log_softmax(output, dim = 1)

        return activatedOutput

In [70]:
model2 = TwoMoonsNet()

In [71]:
import torch.optim as optim

optimizer1 = optim.SGD(model2.parameters(), lr=0.03)
optimizer2 = optim.SGD(model2.parameters(), lr = 0.01)
num_epochs = 10
loss = 0
unsupervised_loss = 0

for epoch in range(num_epochs):
    model2.train()
    
    # Train on labeled data
    for images, labels in selected_loader:
        optimizer1.zero_grad()
        outputs = model2(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer1.step()

    print(f"The supervised loss for epoch{epoch} is:{loss}")
    
    # Train on unlabeled data
    for images, labels in remaining_loader:
        optimizer2.zero_grad()
        outputs = model2(images)
        unsupervised_loss = entropy_loss(outputs)
        unsupervised_loss.backward()
        optimizer2.step()

    
    print(f"The un-supervised loss for epoch{epoch} is:{unsupervised_loss}")

The supervised loss for epoch0 is:0.48841455578804016
The un-supervised loss for epoch0 is:0.5801041126251221
The supervised loss for epoch1 is:0.42923659086227417
The un-supervised loss for epoch1 is:0.5446867942810059
The supervised loss for epoch2 is:0.38170090317726135
The un-supervised loss for epoch2 is:0.5164116621017456
The supervised loss for epoch3 is:0.34158214926719666
The un-supervised loss for epoch3 is:0.4931500256061554
The supervised loss for epoch4 is:0.3071553409099579
The un-supervised loss for epoch4 is:0.4764765202999115
The supervised loss for epoch5 is:0.2772395610809326
The un-supervised loss for epoch5 is:0.46371200680732727
The supervised loss for epoch6 is:0.2506945729255676
The un-supervised loss for epoch6 is:0.45309799909591675
The supervised loss for epoch7 is:0.22713176906108856
The un-supervised loss for epoch7 is:0.4448125660419464
The supervised loss for epoch8 is:0.20651240646839142
The un-supervised loss for epoch8 is:0.4384482800960541
The supervi

In [72]:
train_accuracy = calculate_accuracy(selected_loader, model2)
test_accuracy = calculate_accuracy(remaining_loader, model2)

print(f'Training Accuracy: {train_accuracy}%')
print(f'Testing Accuracy: {test_accuracy}%')

Training Accuracy: 100.0%
Testing Accuracy: 82.97872340425532%
